In [105]:
import pandas as pd
import numpy as np

In [106]:
df = pd.read_csv('../data/pollen_train.csv')
df['location'].unique()
location = 'БЕОГРАД'
path = '../data/pollen_train.csv'
name = location + '_' + path.split('/')[-1]
df.columns
df.head()
df.drop('Unnamed: 0', axis = 1)
"/".join(path.split('/')[:-1]) + '/' + name

df[df['location'] == 'БЕОГРАД - НОВИ БЕОГРАД']

,Unnamed: 0,location,date,ACER,ALNUS,AMBROSIA,ARTEMISIA,BETULA,CANNABACEAE,CARPINUS,...,PLANTAGO,PLATANUS,POACEAE,POPULUS,QUERCUS,RUMEX,SALIX,TILIA,ULMACEAE,URTICACEAE
0,265,БЕОГРАД - НОВИ БЕОГРАД,2016-02-02,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,266,БЕОГРАД - НОВИ БЕОГРАД,2016-02-03,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,0
2,267,БЕОГРАД - НОВИ БЕОГРАД,2016-02-04,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,268,БЕОГРАД - НОВИ БЕОГРАД,2016-02-05,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,269,БЕОГРАД - НОВИ БЕОГРАД,2016-02-06,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9629,33563,БЕОГРАД - НОВИ БЕОГРАД,2021-10-28,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9630,33564,БЕОГРАД - НОВИ БЕОГРАД,2021-10-29,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9631,33565,БЕОГРАД - НОВИ БЕОГРАД,2021-10-30,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9632,33566,БЕОГРАД - НОВИ БЕОГРАД,2021-10-31,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
def split_data(path):
    save_path = "/".join(path.split('/')[:-1]) + '/'
    df = pd.read_csv(path)
    locations = df['location'].unique()
    print(path.split('/')[-1])
    for location in locations:
        #print(location)
        tmp_df = pd.DataFrame()
        name = location + '_' + path.split('/')[-1]
        tmp_df = df[df['location'] == location]
        #tmp_df = df.drop('Unnamed: 0', axis = 1)
        file_name = save_path + name
        #print(file_name)
        tmp_df = tmp_df.drop('location', axis = 1)
        tmp_df.to_csv(file_name) 

In [108]:
split_data('../data/pollen_train.csv')
df = pd.read_csv('../data/ВРШАЦ_pollen_train.csv')
raw_data = [[]]
lst = []

ran = range(1, 1 + 10)
lst.append(np.array(df.iloc[ran]))
lst
df.iloc[[1]]['AMBROSIA']

type((i for i in range(3)))


pollen_train.csv


generator

In [109]:
pd.DataFrame([(df.iloc[1]['AMBROSIA'],df.iloc[2]['AMBROSIA'],df.iloc[3]['AMBROSIA'])] * 10).to_csv('../data/test.csv')

In [116]:
def merge_data(path, target = 'AMBROSIA', stride = 1):
    df = pd.read_csv(path)
    X = []
    y = []
    for i in range(0, df.shape[0] - 13, stride):
        lst = []

        ran = range(i, i + 10)
        X.append(np.array(df.iloc[ran]).flatten())
        y.append((
            float(df.iloc[[i+11]][target]),
            float(df.iloc[[i+12]][target]),
            float(df.iloc[[i+13]][target]),
        ))
    return np.array(X), y


In [117]:
X, y = merge_data('../data/БЕОГРАД - НОВИ БЕОГРАД_pollen_train.csv')
X.shape

(1536, 290)

In [119]:
df = pd.read_csv('../data/pollen_train.csv')
locations = df['location'].unique()

In [120]:
def XGB_input(path, locations, file, target = 'AMBROSIA', stride = 1):
    for location in locations:
        file_name = path + '/' + location + '_' + file
        X, y = merge_data(file_name,target = target, stride = stride)
        pd.DataFrame(X).to_csv(file_name + 'XGB.csv')
        pd.DataFrame(y).to_csv(file_name + 'XGB_target.csv')
    

In [121]:
XGB_input('../data', locations, 'pollen_train.csv')

In [246]:
test1 = pd.read_csv('../data/pollen_train.csv')
weather_df = pd.read_csv('../data/weather_data_filtered.csv')
df = test1.merge(weather_df, how = 'left', on = (['location', 'date']))

In [247]:
df['prcp'] = df['prcp'].interpolate()
df['wspd'] = df['wspd'].interpolate(method = 'slinear')
df['wspd'] = df['wspd'].fillna(0)
df['wdir'] = df['wdir'].interpolate(method = 'slinear')
df['wdir'] = df['wdir'].fillna(0)
df = df.drop(['wpgt','snow','tsun','pres'], axis = 1)
df = df.dropna()


In [248]:
df.to_csv('../data/pollen_train_weather.csv')
split_data('../data/pollen_train_weather.csv')
XGB_input('../data', locations, 'pollen_train_weather.csv')

pollen_train_weather.csv
